---
Lógica Computacional TP1.Exercício2
======================
##### G01 - Bruno Dias da Gião A96544, João Luis da Cruz Pereira A95375
, David Alberto Agra A95726


---

## Exercício 2 - Enunciado

2. Um sistema de tráfego  é representado por um grafo orientado ligado. Os nodos denotam pontos de acesso e  os arcos denotam vias de comunicação só com um sentido .  O grafo tem de ser ligado: entre cada par de nodos $\langle n_1,n_2 \rangle$ tem de existir um caminho $n_1 \leadsto n_2$ e um caminho $n_2\leadsto n_1$.
    1. Gerar aleatoriamente o grafo com  $N \in\{8..15\}$  nodos e com ramos verificando:
        1. Cada nodo tem um número aleatório de descendentes $d\in\{0 .. 3\}\,$ cujos destinos são também gerados aleatoriamente. 
        2. Se  existirem “loops”  ou destinos repetidos, deve-se gerar outro grafo.
    2. Pretende-se fazer  manutenção interrompendo  determinadas vias. Determinar o maior número de vias que é possível remover mantendo o grafo ligado.

---

## Exercício 2 - Solução

### Exercício 2 - Código

### Exercício 2 - Testes